In [95]:

from nltk.corpus import reuters
import re
import spacy
import requests
from bs4 import BeautifulSoup
import re
import random
from spacy.training.example import Example
from spacy import displacy
from spacy.language import Language
import json
import os

## Scrape data set Class

In [105]:
class Scrapping_data:

    def __init__(self):
        pass

    def scrape_prodructs(self):

        #scrapping the oil-based products
        _products = []

        r= requests.get('https://innovativewealth.com/inflation-monitor/what-products-made-from-petroleum-outside-of-gasoline/')
        soup = BeautifulSoup(r.text,'html.parser')
        for i in soup.find_all('td'):
            _products.append(i.string)

        
        #hardcoded products
        _products.append('kerosine')


        #getting the none type out of the list
        _products = filter(None, _products)

        #converting the elements in list to lowercase
        _products = [p.lower() for p in _products]
        
        print(f'products amount= {_products}')
        #saving the data as csv format
        with open('scrapped_oil_products.json','w',encoding='utf-8') as f:
            json.dump(_products,f)
        
        return 'scrapped_oil_products.json'

    def scrape_oil_companies(self):
        #scrapping 220 oil companies
        _com = []
        #page 1 and 3 
        for i in range(1,4):
            url = f'https://companiesmarketcap.com/oil-gas/largest-oil-and-gas-companies-by-market-cap/?page={i}'
            
            r= requests.get(url)
            soup = BeautifulSoup(r.text,'html.parser')
            for j in soup.find_all('div',{'class':'company-name'}):
                _com.append(j.string)

        #lowercase the company names
        _com = [c.lower() for c in _com]
        _com = [c.replace('\n','') for c in _com]

        print(f'companies amount= {len(_com)}')
        #save the data as csv format
        with open('scrapped_oil_companies.json','w',encoding='utf-8') as f:
            json.dump(_com,f)

        return 'scrapped_oil_companies.json'
        
   

    

## Creating the training dataset

In [98]:
class Training_data(object):
    global nlp
    global _com
    global _products 
    
    #initialising the code, needs to recive te nlp object 
    def __init__(self, nlp: Language):
        self.nlp = nlp
        self.scrape = Scrapping_data()
        self._products = self.load_json(self.scrape.scrape_prodructs())
        self._com =  self.load_json(self.scrape.scrape_oil_companies())
        self.other_cate_text = self.making_sentences()
        #getting the training data
        self._other_cat_training_data = []    

        #looping through all the paragraphs
        for i in range(len(self.other_cate_text)):
            #looping through all the sentences
            for j in range(len(self.other_cate_text[i])):
                #checking if sentences have the product name in them
                if self.has_product_name(self.other_cate_text[i][j]):
                    txt = str(self.other_cate_text[i][j])
                    #if they have the product name, we take the sentence and append it to the training data
                    self._other_cat_training_data.append(self.parse_train_data_products(txt))
        
    
        #looping through all the paragraphs
        for i in range(len(self.other_cate_text)):
            #looping through all the sentences
            for j in range(len(self.other_cate_text[i])):
                #checking if sentences have the product name in them
                if self.has_company_name(self.other_cate_text[i][j]):
                    txt = str(self.other_cate_text[i][j])
                    #if they have the product name, we take the sentence and append it to the training data
                    self._other_cat_training_data.append(self.parse_train_data_company(txt))

        self.save_data(self._other_cat_training_data)
        print(len(self._other_cat_training_data))
    
    #save the data
    def save_data(self,data):
        with open('trainig_data.json','w',encoding='utf-8') as f:
            json.dump(data,f)

    #cleaned text using regex, splitting every sentence into elements of a list
    def clean(self, text):
        text = text.replace('&lt;','<') # html escape
        text = re.sub(r'[<>]', '"', text) # quotation marks instead of <>
        text = re.sub(r'[ ]*"[A-Z\.]+"', '', text) # drop stock symbols
        text = re.sub(r'[ ]*\([A-Z\.]+\)', '', text) # drop stock symbols
        text = re.sub(r'\bdlr(s?)\b', r'dollar\1', text, flags=re.I)
        text = re.sub(r'\bmln(s?)\b', r'million\1', text, flags=re.I)
        text = re.sub(r'\bpct\b', r'%', text, flags=re.I)
        text = re.sub(r'"', r'', text) # quotation marks
        text = re.sub(r'\s+', ' ', text) # multiple whitespace by one
        text = re.sub('[()]', '', text)
        text = text.lower()
        arr = []
        arr = text.split('. ')
        return arr
        
    #making the training sentences out of the categories given in the list
    def making_sentences(self):
        #load nlp object
        interesting_categories = ['crude','castor-oil','gas','fuel','nat-gas','oil']

        other_cate_text = []
        for i in interesting_categories:
            reuters_fileids_crudes = reuters.fileids(categories=[i])
            #load raw content of the dataset
            ruw = [reuters.raw(i) for i in reuters_fileids_crudes]

            #splitting everything in sentences(split if there is a . and space)-> ex: 'the end. ' and cleaning the raw text.  
            
            for line in ruw:
                other_cate_text.append(self.clean(line))

        return other_cate_text



        
    #parsing the text into a training format
    def parse_train_data_products(self,txt):
        """
        loops through the product list. if it finds a product in the list that's also in the text, then it'll look for the first char position and the last one. 
        Then it'll asign a PRODUCT entitie to it.
        result and structure example -> ('gasoline supplies would also be limited', {'entities': [(0, 8, 'PRODUCT')]})
        at char index 0 until 8 we have the word 'gasoline' wich is in the product list.
        """
        for pro in self._products:
            if pro in txt:
                start_char = txt.find(pro)
                end_char = txt.find(pro)+len(pro)
                productt = [(start_char,end_char,'PRODUCT')]
                return(txt,{'entities':productt})

                
    #check if doc has product names that were scraped
    def has_product_name(self,text):
        """this function checks of the words are in the product-list.
        """
        doc = self.nlp(text)
        for token in doc:
            #checks of word in product list
            if token.lower_ in self._products:
                if token.pos_ != 'VERB':
                    return True
        return False  


    def parse_train_data_company(self,txt):
        for com in self._com:
            if com in txt:
                start_char = txt.find(com)
                end_char = txt.find(com)+len(com)
                companies = [(start_char,end_char,'COMPANY')]
                return(txt,{'entities':companies})

    #check if doc has the 200 companies that were scrapped
    def has_company_name(self,text):
        doc = self.nlp(text)
        for token in doc:
            if token.lower_ in self._com:
                if token.pos_ != 'VERB':
                    return True
        return False

    #load the json from the scraped class
    def load_json(self,file):
        with open(file,'r',encoding='utf-8')as f:
            data = json.load(f)
        return(data)


## TRAINING LOOP


In [ ]:
class Train_model(object):
    global nlp
    global training_data
    def __init__(self, nlp:Language, training_data:list):
        self.nlp = nlp
        self.training_data= training_data
        self.customizing_pipeline_component()
    
    
    def customizing_pipeline_component(self):
        disabled_pipes = []
        for pipe_name in self.nlp.pipe_names:
            if pipe_name != 'ner':
                self.nlp.disable_pipes(pipe_name)
                disabled_pipes.append(pipe_name)

        
        
        #neural network
        optimizer = self.nlp.create_optimizer()
        #30 iterations, this number is chosen randonly by me
        for i in range(10):
            #every iteration we'll shuffle the training data
            random.shuffle(self.trainig_data)
            losses ={}
            #
            for text, annotations in self.trainig_data:
                
                
                doc = self.nlp.make_doc(text)
                
                ex = Example.from_dict(doc,annotations)
                self.nlp.update([ex],sgd=optimizer, losses=losses)
            print(f'iteration {i} - {losses}')


        for pipe in disabled_pipes:
            self.nlp.enable_pipe(pipe)
        


## Main method where code is initialized

In [103]:
def main():

    nlp = spacy.load('en_core_web_lg') #, disable=['parser', 'tagger'])
    
    #check if data is already is path
    if not os.path.isfile('trainig_data.json'):
        training_data = Training_data(nlp)
        data = training_data
    
    #load data if it is already in path
    else:
        with open('trainig_data.json','r',encoding='utf-8')as f:
            data= json.load(f)
        print('kleir')
    
    
    

if __name__ == '__main__':
    main()


467
